<a href="https://colab.research.google.com/github/yvettegahamanyi/formative2-DataPreprocessing/blob/merge-data_product-recommendation-model/merge_data_product_recommendation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Merging Data The Two  datasets

In [76]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Conv2D, Dense, Activation, MaxPooling2D, Dropout, Flatten
from sklearn.model_selection import train_test_split
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [63]:
customer_social_profiles = pd.read_csv('/content/customer_social_profiles.csv')
customer_transactions = pd.read_csv('/content/customer_transactions.csv')

print(customer_social_profiles.head())
print(customer_transactions.head())

  customer_id_new social_media_platform  engagement_score  \
0            A178              LinkedIn                74   
1            A190               Twitter                82   
2            A150              Facebook                96   
3            A162               Twitter                89   
4            A197               Twitter                92   

   purchase_interest_score review_sentiment  
0                      4.9         Positive  
1                      4.8          Neutral  
2                      1.6         Positive  
3                      2.6         Positive  
4                      2.3          Neutral  
   customer_id_legacy  transaction_id  purchase_amount purchase_date  \
0                 151            1001              408    2024-01-01   
1                 192            1002              332    2024-01-02   
2                 114            1003              442    2024-01-03   
3                 171            1004              256    2024-01-04 

In [64]:
customer_transactions.rename(columns={'customer_id_legacy': 'customer_id_new'}, inplace=True)

In [65]:
customer_social_profiles['customer_id_new'] = customer_social_profiles['customer_id_new'].str.replace('A', '', regex=False).astype(int)

In [66]:
merged_data = pd.merge(customer_social_profiles, customer_transactions, on='customer_id_new', how='inner')
display(merged_data.head())

,customer_id_new,social_media_platform,engagement_score,purchase_interest_score,review_sentiment,transaction_id,purchase_amount,purchase_date,product_category,customer_rating
0,190,Twitter,82,4.8,Neutral,1031,333,2024-01-31,Groceries,3.8
1,190,Twitter,82,4.8,Neutral,1140,401,2024-05-19,Sports,4.9
2,150,Facebook,96,1.6,Positive,1042,389,2024-02-11,Sports,NaN
3,150,Facebook,96,1.6,Positive,1046,177,2024-02-15,Books,3.6
4,162,Twitter,89,2.6,Positive,1079,101,2024-03-19,Books,4.2


In [67]:
merged_data.to_csv('merged_dataset.csv', index=False)

##Product Recommendation Model

In [68]:
df = pd.read_csv('/content/merged_dataset.csv')
print(df.shape)
df.head()

(219, 10)


,customer_id_new,social_media_platform,engagement_score,purchase_interest_score,review_sentiment,transaction_id,purchase_amount,purchase_date,product_category,customer_rating
0,190,Twitter,82,4.8,Neutral,1031,333,2024-01-31,Groceries,3.8
1,190,Twitter,82,4.8,Neutral,1140,401,2024-05-19,Sports,4.9
2,150,Facebook,96,1.6,Positive,1042,389,2024-02-11,Sports,NaN
3,150,Facebook,96,1.6,Positive,1046,177,2024-02-15,Books,3.6
4,162,Twitter,89,2.6,Positive,1079,101,2024-03-19,Books,4.2


In [69]:
feature_columns = [
    'engagement_score',
    'purchase_interest_score',
    'purchase_amount',
    'customer_rating'
]

# Add demographic or other useful features if present:
optional_cols = [
    'age', 'monthly_visits', 'total_transactions'
]

for col in optional_cols:
    if col in df.columns:
        feature_columns.append(col)

print("Using features:", feature_columns)

Using features: ['engagement_score', 'purchase_interest_score', 'purchase_amount', 'customer_rating']


In [70]:
X = df[feature_columns]
y = df['product_category']


In [71]:
encoder = LabelEncoder()
y_enc = encoder.fit_transform(y)

**Train Test Split**

In [74]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc,
    test_size=0.2,
    random_state=42,
    stratify=y_enc
)

**Scale The Features**

In [77]:
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

**Training The Model**

In [79]:
model = RandomForestClassifier(
    n_estimators=300,
    max_depth=25,
    class_weight="balanced",
    random_state=42
)

model.fit(X_train_s, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=25, n_estimators=300,
                       random_state=42)

**Evaluate**

In [80]:
y_pred = model.predict(X_test_s)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=encoder.classes_))

Accuracy: 0.5454545454545454

Classification Report:
              precision    recall  f1-score   support

       Books       0.60      0.75      0.67         8
    Clothing       1.00      0.57      0.73         7
 Electronics       0.38      0.33      0.35         9
   Groceries       1.00      0.62      0.77         8
      Sports       0.35      0.50      0.41        12

    accuracy                           0.55        44
   macro avg       0.67      0.56      0.59        44
weighted avg       0.62      0.55      0.56        44



**Saving The Model**

In [81]:
bundle = {
    "model": model,
    "scaler": scaler,
    "encoder": encoder,
    "feature_columns": feature_columns
}

joblib.dump(bundle, "product_recommendation_model.joblib")
print("Saved product_recommendation_model.joblib")

Saved product_recommendation_model.joblib
